In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
from pprint import pprint
import requests
import json
import data_import

In [2]:
rest_df = data_import.get_rest_countries()

In [4]:
rest_df.to_csv("rest_df.csv")